In [ ]:
#
# Import Packages
#
import sliderule
from sliderule import icesat2
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pyproj
import re
import os

In [ ]:
icesat2.init("slideruleearth.io", verbose=True, organization="developers", desired_nodes=7)

In [ ]:
#
#  Build Region of Interest
#
xy0=np.array([  -73000., -2683000.])
transformer = pyproj.Transformer.from_crs(3413, 4326)
xyB=[xy0[0]+np.array([-1, 1, 1, -1, -1])*1.e4, xy0[1]+np.array([-1, -1, 1, 1, -1])*1.e4]
llB=transformer.transform(*xyB)
poly=[{'lat':lat,'lon':lon} for lat, lon in zip(*llB)]
plist = []
for p in poly:
    plist += p["lat"], 
    plist += p["lon"],
region_of_interest = sliderule.toregion(plist)
region_of_interest["gdf"].plot()

In [ ]:
#
# Make Processing Request
#
parms = { "poly": region_of_interest["poly"],
          "cnf": "atl03_high",
          "ats": 10.0,
          "cnt": 5,
          "len": 40.0,
          "res": 120.0,
          "maxi": 5,
          "cycle": 15,
          "rgt": 658,
          "time_start":'2020-01-01',
          "time_end":'2021-01-01',
          "samples": {"strips": {"asset": "arcticdem-strips", "with_flags": True}} }
gdf = icesat2.atl06p(parms, asset="nsidc-s3")

In [ ]:
#
# Set DEM of Interest
#
id = 17
gdf.attrs['file_directory']

In [ ]:
#
# Functions to Pull Out Bounding Box of Raster
#
def getXY(line):
    line = line.replace("(","$")
    line = line.replace(")","$")
    point = line.split("$")[1]
    coord = point.split(",")
    x = float(coord[0].strip())
    y = float(coord[1].strip())
    return x, y

def getLonLat(line):
    line = line.replace("(","$")
    line = line.replace(")","$")
    point = line.split("$")[3]
    coord = point.split(",")
    deg, minutes, seconds, direction = re.split('[d\'"]', coord[1].strip())
    lon = (float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction in ['W', 'S'] else 1)
    deg, minutes, seconds, direction = re.split('[d\'"]', coord[0].strip())
    lat = (float(deg) + float(minutes)/60 + float(seconds)/(60*60)) * (-1 if direction in ['W', 'S'] else 1)
    return [lon, lat]

def getBB(dem):
    os.system("gdalinfo /vsis3/{} > /tmp/r.txt".format(dem))
    with open("/tmp/r.txt", "r") as file:
        lines = file.readlines()
        for line in lines:
            if "Upper Left" in line:
                ul = getLonLat(line)
            elif "Lower Left" in line:
                ll = getLonLat(line)
            elif "Upper Right" in line:
                ur = getLonLat(line)
            elif "Lower Right" in line:
                lr = getLonLat(line)
    return ul + ll + lr + ur + ul

#
# Get Boundaries for each Raster
#
raster_of_interest = {}
for i in gdf.attrs['file_directory']:
    rlist = getBB(gdf.attrs['file_directory'][i])
    raster_of_interest["dem"+str(i)] = sliderule.toregion(rlist)

In [ ]:
#
# Pull Out DEM Values
#
def getValue(x, file_id):
    l = np.where(x['strips.file_id'] == file_id)[0]
    if len(l) == 1:
        return x['strips.value'][l[0]]
    else:
        return None
sampled_data = gdf[gdf['strips.time'].notnull()]
for i in gdf.attrs['file_directory']:
    sampled_data["dem"+str(i)] = sampled_data.apply(lambda x: getValue(x, i), axis=1)

In [ ]:

#
# Create Boundaries for Region and Raster
#

#
# Plot
#
fig = plt.figure(num=None, figsize=(24, 24))
region_lons = [p["lon"] for p in region_of_interest["poly"]]
region_lats = [p["lat"] for p in region_of_interest["poly"]]
ax = {}
for i in gdf.attrs['file_directory']:
    raster_lons = [p["lon"] for p in raster_of_interest["dem"+str(i)]["poly"]]
    raster_lats = [p["lat"] for p in raster_of_interest["dem"+str(i)]["poly"]]
    plot_data = sampled_data[sampled_data["dem"+str(i)].notnull()]
    plot_data = sampled_data[sampled_data["dem"+str(i)] > -9990]
    ax[i] = plt.subplot(5,4,i+1)
    gdf.plot(ax=ax[i], column='h_mean', color='y', markersize=0.5)
    plot_data.plot(ax=ax[i], column='h_mean', color='b', markersize=0.5)
    ax[i].plot(region_lons, region_lats, linewidth=1.5, color='r', zorder=2)
    ax[i].plot(raster_lons, raster_lats, linewidth=1.5, color='g', zorder=2)
plt.tight_layout()

In [ ]:
#
# Plot the Different ArcticDEM Values against the SlideRule ATL06-SR Values
#

######################
# Select DEM File ID #
######################
file_id = 5

# Setup Plot
fig,ax = plt.subplots(num=None, figsize=(10, 8))
ax.set_title("SlideRule vs. ArcticDEM Elevations")
ax.set_xlabel('distance (m)')
ax.set_ylabel('height (m)')
legend_elements = []

# Filter Data to Plot
plot_data = sampled_data[sampled_data["dem"+str(file_id)].notnull()]
plot_data = sampled_data[sampled_data["dem"+str(file_id)] > -9990]

# Set X Axis
x_axis = plot_data["distance"]

# Plot SlideRule ATL06 Elevations
sc1 = ax.scatter(x_axis, plot_data["h_mean"].values, c='red', s=2.5)
legend_elements.append(matplotlib.lines.Line2D([0], [0], color='red', lw=6, label='ATL06-SR'))

# Plot ArcticDEM Elevations
sc2 = ax.scatter(x_axis, plot_data["dem"+str(file_id)].values, c='blue', s=2.5)
legend_elements.append(matplotlib.lines.Line2D([0], [0], color='blue', lw=6, label='ArcticDEM'))

# Display Legend
lgd = ax.legend(handles=legend_elements, loc=3, frameon=True)
lgd.get_frame().set_alpha(1.0)
lgd.get_frame().set_edgecolor('white')

# Show Plot
plt.show()